In [2]:
import stageemi.dev.decorator_map as dm
import xarray as xr 
import ipywidgets as widg 
import ipyleaflet as ipyl
import datetime as dt
import os 
import numpy as np
import shapely
from stageemi.dev.visu import Borders
import geojson

In [3]:
#fname = "../weatherdata/arome_2m_NW_20180501000000.grib"
#fname = "/scratch/labia/chabotv/TestEAS/FileList/20191104000000__PAROME__T__EURW1S100______GRILLE____0_48_1__HAUTEUR__2__.nc"

var2plot = "WWMF"
#var2plot = "DD"
dir_in = '/scratch/labia/lepapeb/StageEMI/'+var2plot+'/'
if var2plot == 'DD':
    fname  = dir_in + '20200122060000__PAROME__'+var2plot+'__EURW1S100______GRILLE____0_48_1__HAUTEUR__10__.nc'

if var2plot == 'WWMF':
    fname  = dir_in + '20200122060000__PG0PAROME__'+var2plot+'__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc'
    
fname  = dir_in + '20200122060000__PG0PAROME__'+var2plot+'__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc'
ds = xr.open_dataset(fname)
print(fname)

/scratch/labia/lepapeb/StageEMI/WWMF/20200122060000__PG0PAROME__WWMF__EURW1S100______GRILLE____0_48_1__SOL____GRIB2.nc


In [4]:
fname_mask = '/home/mrpa/borderiesm/stageEMI/Codes/stageemi/stageemi/GeoData/nc_departement/FRB05.nc'
ds_mask = xr.open_dataset(fname_mask)
ds_mask["latitude"]=ds_mask["latitude"].round(5)
ds_mask["longitude"]=ds_mask["longitude"].round(5)
ds_mask["latitude"].values.mean()

47.64999999999999

In [5]:


ds['latitude']=ds['latitude'].round(5)
ds['longitude']=ds['longitude'].round(5)
#ds_mask['latitude']
#ds['latitude']
ds_b = ds.load()*ds_mask["mask"].isel(id=0).load()
ds_b
#ds_b['unknown'].isel(step=0).plot()
#ds_mask["mask"].plot()
#ds['unknown'].isel(step=0).plot() # pour ploter
#ds_d

<xarray.Dataset>
Dimensions:            (latitude: 93, longitude: 164, step: 42)
Coordinates:
  * latitude           (latitude) float64 48.11 48.1 48.09 ... 47.21 47.2 47.19
  * longitude          (longitude) float64 0.61 0.62 0.63 ... 2.22 2.23 2.24
  * step               (step) timedelta64[ns] 01:00:00 ... 1 days 18:00:00
    surface            int64 0
    heightAboveGround  int64 2
    id                 <U5 'FRB05'
Data variables:
    unknown            (step, latitude, longitude) float32 nan nan ... nan nan

In [6]:
#latmin, latmax = 38,42
#lonmin, lonmax = 0.,4.
#lat = ds['latitude'].values
#lon = ds['longitude'].values
#ds_b = ds.sel(latitude=lat[(lat>=latmin)&(lat>=latmax)]).sel(longitude = lon[(lon>=lonmin)&(lon<=lonmax)])
#ds_b


In [6]:
@dm.gogeojson_wwmf(type='Pixel')
def get_step(step):
    return ds_b["unknown"].isel(step=step)

def update_html(feature, **kwargs):
    html1.value = '''
        <h4> Type de temps </h4>
        <h4><b>{}</b></h4>
    '''.format(feature['properties']['value'])
    
def on_change(change):
    geo_file,legend_file = get_step(change["new"])
    geojson_layer = ipyl.GeoJSON(data=geo_file,hover_style={"opacity":1})
    if len(m.layers) == 1:
        m.add_layer(geojson_layer)
    else: 
        m.substitute_layer(m.layers[1],geojson_layer)
    legend_file.seek(0)
    legend.value =legend_file.read() 
#print(ds_b.isel(step=2))
   
slider = widg.IntSlider(min=0,max=len(ds_b.step),step=1,value=0,description="step")
slider.observe(on_change,'value')
ds_b

<xarray.Dataset>
Dimensions:            (latitude: 93, longitude: 164, step: 42)
Coordinates:
  * latitude           (latitude) float64 48.11 48.1 48.09 ... 47.21 47.2 47.19
  * longitude          (longitude) float64 0.61 0.62 0.63 ... 2.22 2.23 2.24
  * step               (step) timedelta64[ns] 01:00:00 ... 1 days 18:00:00
    surface            int64 0
    heightAboveGround  int64 2
    id                 <U5 'FRB05'
Data variables:
    unknown            (step, latitude, longitude) float32 nan nan ... nan nan

In [7]:
m = ipyl.Map(center=(40,1),zoom=8)
m.add_control(ipyl.WidgetControl(widget=slider,position="topright"))
geo_file,legend_file = get_step(0)
geojson_layer = ipyl.GeoJSON(data=geo_file,hover_style={"opacity":1})
m.add_layer(geojson_layer)

/opt/anaconda3/envs/preproc2/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [8]:
legend_file.seek(0)
legend = widg.Image(layout=widg.Layout(height="430px"))
legend.value = legend_file.read() 
widg_map = widg.HBox([m,legend])
#m.add

In [44]:
Base_Dept = '/home/mrpa/borderiesm/stageEMI/Codes/stageemi/stageemi/GeoData/Departements/'
Base_marine = '/home/mrpa/borderiesm/stageEMI/Codes/stageemi/stageemi/GeoData/ZoneLargeMarine/'
border_reg = Borders(Base_Dept + "Herault.json", color="red", layer_name="Herault")
border_med = Borders(Base_marine+'ZonesMarineLarges_Mediterrannee.json',color = 'blue',layer_name='Mediterrannee')

m.add_layer(border_reg)
m.add_layer(border_med)

In [9]:
widg_map